Import libary

In [201]:
import os
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [202]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from tensorflow.keras import layers,Sequential,datasets,layers,optimizers,Sequential,metrics
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model

**1.ResNet20 & Data Prepossessing**

In [203]:
(train_x, train_y), (test_x, test_y) = cifar10.load_data()
#train_x = train_x.astype("float32") / 255
#test_x = test_x.astype("float32") / 255
train_x, test_x = train_x / 255.0, test_x / 255.0
print(type(train_x))
print(type(test_x))

In [204]:
train_x = train_x[:10000]
train_y = train_y[:10000]
print(train_x.shape)
print(train_y.shape)

In [206]:
##  Convert to one-hot encoding
def change_one_hot_label(X):
    Y = np.zeros((X.size, 10))
    for idx, row in enumerate(Y):
        row[X[idx]] = 1.0

    return Y

In [207]:
num_classes = 10
train_Y = change_one_hot_label(train_y)
test_Y = change_one_hot_label(test_y)
plt.imshow(train_x[1])
print(train_Y[1])

In [208]:
plt.imshow(test_x[1])
print(test_Y[1])

In [209]:
# z-normalize data
# normalize train set
train_x_std = np.std(train_x)
train_X = np.nan_to_num((train_x - np.mean(train_x,axis=0))/train_x_std) 
print(np.mean(train_X))
print(np.std(train_X))

# normalize test set
test_x_std = np.std(test_x)
test_X = np.nan_to_num((test_x - np.mean(test_x,axis=0))/test_x_std) 

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

In [210]:
labels = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
plt.figure(figsize=(14,14)) 
for i in range(10):   
    plt.subplot(4,5,i+1)
    plt.grid(False)
    plt.imshow(train_x[i])
    t = labels[train_y[i][0]]
    plt.title(t) 
plt.show()

In [12]:
plt.figure(figsize=(14,14)) 
for i in range(10):   
    plt.subplot(4,5,i+1)
    plt.grid(False)
    plt.imshow(test_x[i])
    t = labels[test_y[i][0]]
    plt.title(t) 
plt.show()

In [211]:
from keras.models import Model, load_model
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

In [14]:
def ResNet20():
    inputs = layers.Input(shape=(32, 32, 3))
    x = layers.Conv2D(16, 3, padding="same",
                      kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    num_filters = 16
    for stack in range(3):
        for res_block in range(3):
            if stack > 0 and res_block == 0:
                shortcut = layers.Conv2D(num_filters, 1, strides=2, padding="same",
                                         kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(x)
                x = layers.Conv2D(num_filters, 3, strides=2, padding="same",
                                  kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(x)
            else:
                shortcut = x
                x = layers.Conv2D(num_filters, 3, padding="same",
                                  kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(x)
            x = layers.BatchNormalization()(x)
            x = layers.Activation("relu")(x)
            x = layers.Conv2D(num_filters, 3, padding="same",
                              kernel_regularizer=l2(0.0001), kernel_initializer='he_normal')(x)
            x = layers.BatchNormalization()(x)
            x = layers.Add()([shortcut, x])
            x = layers.Activation("relu")(x)
        num_filters *= 2
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(10, kernel_initializer='he_normal', activation="softmax")(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
model = ResNet20()
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['acc'])
model.summary()

In [16]:
checkpoint = ModelCheckpoint(filepath="models/ResNet20.h5", monitor='val_acc',
                             verbose=1, save_best_only=True)

In [22]:
History = model.fit(train_X, train_Y, batch_size=64,
                    validation_data=(test_X, test_Y),
                    epochs=100, verbose=1, callbacks=[checkpoint])

In [24]:
# Accuracy over training time
plt.plot(History.history['acc'], label="Training Accuracy")
plt.plot(History.history['val_acc'], label="Validation Accuracy")
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Loss over training time
plt.plot(History.history['loss'], label="Training Loss")
plt.plot(History.history['val_loss'], label="Validation Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [25]:
print(np.max(History.history["val_acc"]))

2. Mixup

In [294]:
import numpy as np
print(np.random.beta(0.2,0.2))
print(np.random.beta(0.4,0.4))

In [295]:
print(train_x.shape[0])
a = np.random.choice(train_x.shape[0],size=1,replace=True)
print(a)
print([a[0]])

In [330]:
def mixup(x1, y1, alpha,x2,y2):
# x = lambda*x_i+(1-lambda)*x_j
    lam = np.random.beta(alpha, alpha) #lambda
    img = lam * x1 + (1. - lam) * x2
    lab = lam * y1 + (1. - lam) * y2
        
    
    return img,lab

alpha = 0.2

In [331]:
choice = np.empty(10000,int)
train_x_m1 = np.empty((10000, 32, 32, 3))
train_y_m1 = np.empty((10000, 10))
choice_x = np.empty((10000, 32, 32, 3))
choice_y = np.empty((10000, 10))


for i in range (10000):
    choice[i] =  np.random.choice(10000,size=1,replace=True)
    choice_x[i] = train_X[choice[i]] 
    choice_y[i] = train_Y[choice[i]] 
    train_x_m1[i],train_y_m1[i] = mixup (train_X[i], train_Y[i], 0.2,choice_x[i],choice_y[i])
    
    
plt.imshow(train_x_m1[5])
print(train_y_m1[5])

In [332]:
model = ResNet20()
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['acc'])
model.summary()

In [333]:
checkpoint = ModelCheckpoint(filepath="models/ResNet20_m1.h5", monitor='val_acc',
                             verbose=1, save_best_only=True)

In [301]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=False)
datagen.fit(train_x_m1)

In [302]:
History = model.fit_generator(datagen.flow(train_x_m1, train_y_m1, batch_size=64),
                    validation_data=(test_X, test_Y),
                    epochs=100, verbose=1, callbacks=[checkpoint])

In [305]:
# Accuracy over training time
plt.plot(History.history['acc'], label="Training Accuracy")
plt.plot(History.history['val_acc'], label="Validation Accuracy")
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Loss over training time
plt.plot(History.history['loss'], label="Training Loss")
plt.plot(History.history['val_loss'], label="Validation Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [306]:
print(np.max(History.history["val_acc"]))

alpha = 0.4

In [340]:
choice = np.empty(10000,int)
train_x_m2 = np.empty((10000, 32, 32, 3))
train_y_m2 = np.empty((10000, 10))
choice_x = np.empty((10000, 32, 32, 3))
choice_y = np.empty((10000, 10))


for i in range (10000):
    choice[i] =  np.random.choice(10000,size=1,replace=True)
    choice_x[i] = train_X[choice[i]] 
    choice_y[i] = train_Y[choice[i]] 
    train_x_m2[i],train_y_m2[i] = mixup (train_X[i], train_Y[i], 0.2,choice_x[i],choice_y[i])
    
    
plt.imshow(train_x_m2[5])
print(train_y_m2[5])

In [308]:
model = ResNet20()
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['acc'])
model.summary()

In [310]:
checkpoint = ModelCheckpoint(filepath="models/ResNet20_m2.h5", monitor='val_acc',
                             verbose=1, save_best_only=True)

In [311]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=False)
datagen.fit(train_x_m2)

In [312]:
History = model.fit_generator(datagen.flow(train_x_m2, train_y_m2, batch_size=64),
                    validation_data=(test_X, test_Y),
                    epochs=100, verbose=1, callbacks=[checkpoint])

In [313]:
# Accuracy over training time
plt.plot(History.history['acc'], label="Training Accuracy")
plt.plot(History.history['val_acc'], label="Validation Accuracy")
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Loss over training time
plt.plot(History.history['loss'], label="Training Loss")
plt.plot(History.history['val_loss'], label="Validation Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [314]:
print(np.max(History.history["val_acc"]))

### 3.Cutout

In [184]:
A = np.random.randint(2, size=10)
print(A)

In [185]:
    def cutout(img,F):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.shape[1]
        w = img.shape[0]
     #print(h)
      # print(w)
        K  =16
        num_holes = 1
        mask = np.ones((h, w,3), np.float32)
       #print(mask.shape)
        
        for n in range(num_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - K // 2, 0, h)
            y2 = np.clip(y + K // 2, 0, h)
            x1 = np.clip(x - K // 2, 0, w)
            x2 = np.clip(x + K // 2, 0, w)

            mask[y1: y2, x1: x2, :] = 0.
       #print(mask.shape)
   
        img1 = img * mask
    
        if F == 1:
            return img1
        else:
            return img
        

In [186]:
train_x_c = np.empty((10000, 32, 32, 3))
train_y_c = np.empty((10000, 10))
F = np.empty(10000)

for l in range (10000):
    F[l] =  np.random.randint(0, 2)
    train_x_c[l] = cutout(train_X[l],F[l])
    train_y_c[l] = train_Y[l]
    
plt.imshow(train_x_c[5])
print(train_y_c[5])

In [187]:
model = ResNet20()
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['acc'])
model.summary()

In [188]:
checkpoint = ModelCheckpoint(filepath="models/ResNet20_c.h5", monitor='val_acc',
                             verbose=1, save_best_only=True)

datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=False)
datagen.fit(train_x_c)

In [189]:
History = model.fit_generator(datagen.flow(train_x_c, train_y_c, batch_size=64),
                    validation_data=(test_X, test_Y),
                    epochs=100, verbose=1, callbacks=[checkpoint])

In [190]:
# Accuracy over training time
plt.plot(History.history['acc'], label="Training Accuracy")
plt.plot(History.history['val_acc'], label="Validation Accuracy")
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Loss over training time
plt.plot(History.history['loss'], label="Training Loss")
plt.plot(History.history['val_loss'], label="Validation Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [358]:
print(np.max(History.history["val_acc"]))

4.Standard

In [233]:
import cv2

def horizontal_flip(img):

    flip_out = cv2.flip(img, 1)

    return flip_out


In [234]:
def random_shift(img,F):
    K = 4
    out = np.zeros((32, 32, 3))
    ##integers
    k1 = np.random.randint(-K, K)
    k2 = np.random.randint(-K, K)
    
    #upward(>0)
   
    if k1 > 0:
        for i in range(32-k1):
            for j in range(32):

                out[i][j][:] = img[i+k1][j][:]
    else:
    # downward(<0)
        for i in range(-k1,32):
            
            for j in range(32):
                out[i][j][:] = img[i+k1][j][:]
    
    # rightward
    if k2 > 0:
        for i in range(32):
            for j in range(32-k2):

                out[i][j][:] = img[i][j+k2][:]
    else:
    #leftward    
        for i in range(32):
            
            for j in range(-k2,32):
                out[i][j][:] = img[i][j+k2][:]
                
    if F == 1:
        flip_out = horizontal_flip(out)
        return flip_out
    else:
        return out

In [236]:
F = np.empty(10000)
train_x_s = np.empty((10000,32,32,3))
train_y_s = np.empty((10000,10))
for i in range(10000):
    F[i] =  np.random.randint(0, 2)
    train_x_s[i] = random_shift(train_X[i],F[i])
    train_y_s[i] = train_Y[i]
plt.imshow(train_x_s[5])
print(train_y_s[5])

In [239]:
plt.imshow(train_X[5])

In [240]:
model = ResNet20()
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['acc'])
model.summary()

In [241]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=False)
datagen.fit(train_x_s)

In [242]:
checkpoint = ModelCheckpoint(filepath="models/ResNet20_s.h5", monitor='val_acc',
                             verbose=1, save_best_only=True)

In [243]:
History = model.fit_generator(datagen.flow(train_x_s, train_y_s, batch_size=64),
                    validation_data=(test_X, test_Y),
                    epochs=100, verbose=1, callbacks=[checkpoint])

In [244]:
# Accuracy over training time
plt.plot(History.history['acc'], label="Training Accuracy")
plt.plot(History.history['val_acc'], label="Validation Accuracy")
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Loss over training time
plt.plot(History.history['loss'], label="Training Loss")
plt.plot(History.history['val_loss'], label="Validation Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [245]:
print(np.max(History.history["val_acc"]))
print(np.max(History.history["acc"]))

5. combine

In [341]:
import cv2

In [342]:
 def combine(img,F,):

## mixup
        h = img.shape[1]
        w = img.shape[0]
     #print(h)
      # print(w)
        K = 16
        num_holes = 1
        mask = np.ones((h, w,3), np.float32)
       #print(mask.shape)
        
        for n in range(num_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - K // 2, 0, h)
            y2 = np.clip(y + K // 2, 0, h)
            x1 = np.clip(x - K // 2, 0, w)
            x2 = np.clip(x + K // 2, 0, w)

            mask[y1: y2, x1: x2, :] = 0.
       #print(mask.shape)
   
        img1 = img * mask
    
## cutout    

        L = 4
        out = np.zeros((32, 32, 3))
        ##integers
        k1 = np.random.randint(-L, L)
        k2 = np.random.randint(-L, L)

        #upward(>0)

        if k1 > 0:
            for i in range(32-k1):
                for j in range(32):

                    out[i][j][:] = img[i+k1][j][:]
        else:
        # downward(<0)
            for i in range(-k1,32):

                for j in range(32):
                    out[i][j][:] = img[i+k1][j][:]

        # rightward
        if k2 > 0:
            for i in range(32):
                for j in range(32-k2):

                    out[i][j][:] = img[i][j+k2][:]
        else:
        #leftward    
            for i in range(32):

                for j in range(-k2,32):
                    out[i][j][:] = img[i][j+k2][:]
                
        out = cv2.flip(out, 1)
    
        if F == 1:
            return img1
        else:
            return out

In [346]:
F = np.empty(10000)
train_x_com = np.empty((10000,32,32,3))
train_y_com = np.empty((10000,10))
for i in range(10000):
    F[i] =  np.random.randint(0, 2)
    train_x_com[i] = combine(train_X[i],F[i])
    train_y_com[i] = train_Y[i]

In [347]:
def mixup(x1, y1, alpha,x2,y2):
# x = lambda*x_i+(1-lambda)*x_j
    lam = np.random.beta(alpha, alpha) #lambda
    img = lam * x1 + (1. - lam) * x2
    lab = lam * y1 + (1. - lam) * y2
        
    return img,lab

In [348]:
choice = np.empty(10000,int)
train_x_comb = np.empty((10000, 32, 32, 3))
train_y_comb = np.empty((10000, 10))
choice_x = np.empty((10000, 32, 32, 3))
choice_y = np.empty((10000, 10))


for i in range (10000):
    choice[i] =  np.random.choice(10000,size=1,replace=True)
    choice_x[i] = train_x_com[choice[i]] 
    choice_y[i] = train_y_com[choice[i]] 
    train_x_comb[i],train_y_comb[i] = mixup (train_x_com[i], train_y_com[i], 0.4, choice_x[i], choice_y[i])
    
    
plt.imshow(train_x_comb[5])
print(train_y_comb[5])

In [349]:
model = ResNet20()
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['acc'])
model.summary()

In [350]:
checkpoint = ModelCheckpoint(filepath="models/ResNet20_comb.h5", monitor='val_acc',
                             verbose=1, save_best_only=True)

In [351]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=False)
datagen.fit(train_x_comb)

In [352]:
History = model.fit_generator(datagen.flow(train_x_comb, train_y_comb, batch_size=64),
                    validation_data=(test_X, test_Y),
                    epochs=100, verbose=1, callbacks=[checkpoint])

In [353]:
# Accuracy over training time
plt.plot(History.history['acc'], label="Training Accuracy")
plt.plot(History.history['val_acc'], label="Validation Accuracy")
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Loss over training time
plt.plot(History.history['loss'], label="Training Loss")
plt.plot(History.history['val_loss'], label="Validation Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [356]:
print(np.max(History.history["val_acc"]))
print(np.max(History.history["acc"]))

Q6. comment

In [ ]:
# Accuracy over training time
plt.plot(History.history['acc'], label="Training Accuracy")
plt.plot(History.history['val_acc'], label="Validation Accuracy")
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Loss over training time
plt.plot(History.history['loss'], label="Training Loss")
plt.plot(History.history['val_loss'], label="Validation Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()